# Test wind function

In [2]:
import sys
sys.path.append('../code')
import importlib

import numpy as np
import pandas as pd

import SeaLevelContrib as slc

In [3]:
importlib.reload(slc)

<module 'SeaLevelContrib' from '/Users/dewilebars/Projects/SLBudget/notebooks/../code/SeaLevelContrib.py'>

## This is the old function that allows to take a list of tide gauges in input

In [16]:
def make_wpn_ef_old(tg_id, tgm_df, with_nodal, with_trend, product):
    
    for i in range( len(tg_id)):
        tg_lat, tg_lon = slc.tg_lat_lon(tg_id[i])
        annual_wind_df = slc.make_wind_df(tg_lat, tg_lon, product)
        df_c = tgm_df.join(annual_wind_df, how='inner')
        df_c.index.names = ['year']
        linear_fit, names = slc.linear_model_zsm(df_c, with_trend, with_nodal, 
                                             with_wind=True, with_pres=True, with_ar=False)
        time_y = df_c.index
        mod = np.array(linear_fit.params[:]) * np.array(linear_fit.model.exog[:,:])
        w_ef = mod[:,[1,2]].sum(axis=1)
        p_ef = mod[:,3]
        
        if with_nodal:
            n_ef = mod[:,[4,5]].sum(axis=1)
            
        if i==0:
            if with_nodal:
                n_ef_df = pd.DataFrame(data=dict(time=time_y, col_name=n_ef))
                n_ef_df = n_ef_df.set_index('time')
                n_ef_df.columns  = [str(tg_id[i])] 
            w_ef_df = pd.DataFrame(data=dict(time=time_y, col_name=w_ef))
            w_ef_df = w_ef_df.set_index('time')
            w_ef_df.columns  = [str(tg_id[i])]
            p_ef_df = pd.DataFrame(data=dict(time=time_y, col_name=p_ef))
            p_ef_df = p_ef_df.set_index('time')
            p_ef_df.columns  = [str(tg_id[i])]
        else:
            if with_nodal:
                n_ef_df[str(tg_id[i])] = n_ef
            w_ef_df[str(tg_id[i])] = w_ef
            p_ef_df[str(tg_id[i])] = p_ef
            
    if with_nodal:
        wpn_ef_df = pd.DataFrame(data=dict(time=time_y, Nodal=n_ef_df.mean(axis=1), 
                        Wind=w_ef_df.mean(axis=1), Pressure=p_ef_df.mean(axis=1)))
    else:
        wpn_ef_df = pd.DataFrame(data=dict(time=time_y, 
                Wind=w_ef_df.mean(axis=1), Pressure=p_ef_df.mean(axis=1)))
        
    wpn_ef_df = wpn_ef_df.set_index('time')
    
    return wpn_ef_df

In [13]:
tg_id = [20]

tgm_df = slc.tide_gauge_obs(tg_id, interp=False)
tgm_df = tgm_df['Average'].to_frame(name='height').dropna()

tgm_df

,height
time,
1862,-2.450625
1863,-4.350625
1864,-7.850625
1865,-4.950625
1866,2.249375
...,...
2017,16.349375
2018,11.849375
2019,16.849375


In [17]:
with_nodal = False
with_trend = True
product = 'ERA5'

wp_old_df = make_wpn_ef_old(tg_id, tgm_df, with_nodal, with_trend, product)

found point 51.5 3.5


In [19]:
wp_old_df

,Wind,Pressure
time,,
1950,3.377823,0.534535
1951,1.456953,1.177441
1952,2.285935,0.446929
1953,1.917413,-1.452489
1954,5.875192,0.669053
...,...,...
2017,3.964652,-0.423774
2018,1.141289,0.004970
2019,2.540051,0.378053


## New function only takes the coordinates as input

In [30]:
def make_wpn_ef(coord, tgm_df, with_nodal, with_trend, product):
    
    annual_wind_df = slc.make_wind_df(coord[0], coord[1], product)
    df_c = tgm_df.join(annual_wind_df, how='inner')
    df_c.index.names = ['year']
    linear_fit, names = slc.linear_model_zsm(df_c, with_trend, with_nodal, 
                                         with_wind=True, with_pres=True, with_ar=False)
    
    mod = np.array(linear_fit.params[:]) * np.array(linear_fit.model.exog[:,:])
    
    wpn_ef_df = pd.DataFrame(index=df_c.index)
    
    if with_nodal:
        wpn_ef_df['Nodal'] = mod[:,[4,5]].sum(axis=1)
    
    
    wpn_ef_df['Wind'] = mod[:,[1,2]].sum(axis=1)
    wpn_ef_df['Pressure'] = mod[:,3]
    
    return wpn_ef_df

In [33]:
coord = slc.tg_lat_lon(tg_id[0])

wp_df = make_wpn_ef(coord, tgm_df, False, with_trend, product)

wp_df

found point 51.5 3.5


,Wind,Pressure
year,,
1950,3.377823,0.534535
1951,1.456953,1.177441
1952,2.285935,0.446929
1953,1.917413,-1.452489
1954,5.875192,0.669053
...,...,...
2017,3.964652,-0.423774
2018,1.141289,0.004970
2019,2.540051,0.378053
